In [1]:
from datasets import load_dataset, Dataset, Features, Image, Value
from pathlib import Path
import json

from huggingface_hub import HfApi

/Users/bitan/Desktop/medgemma/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd dataset_treatment_classified/

/Users/bitan/Desktop/medgemma/dataset_treatment_classified


/Users/bitan/Desktop/medgemma/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
records = []
for line in Path("dataset.jsonl").read_text().splitlines():
    item = json.loads(line)
    records.append({
        "filename": item["filename"],                     # provided image
        "tooth": item["tooth"],
        "treatment": item["treatment"],
        "diagnosis": item["diagnosis"],
        "text": item["diagnosis"] +"; recommended treatment: " +item["treatment"] 
    })

# Define features explicitly
features = Features({
    "filename": Image(mode="RGB"),
    "tooth": Value("string"),
    "treatment": Value("string"),
    "diagnosis": Value("string"),
    "text": Value("string")
})


In [4]:
print(json.dumps(records[0], indent=2))

{
  "filename": "extraction_img26_tooth_00000.jpg",
  "tooth": "first_premolar",
  "treatment": "none",
  "diagnosis": "Tooth appears normal",
  "text": "Tooth appears normal; recommended treatment: none"
}


In [5]:
ds = Dataset.from_list(records, features=features)

In [6]:
print(ds.features)

{'filename': Image(mode='RGB', decode=True), 'tooth': Value('string'), 'treatment': Value('string'), 'diagnosis': Value('string'), 'text': Value('string')}


In [7]:
ds.push_to_hub(
    repo_id="justacoderwhocodes/dental_diagnosis_balanced_dataset",
    split="train",
    private=True,
    commit_message="Closeup dataset"
)


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 19.21ba/s]
Processing Files (1 / 1): 100%|██████████| 45.9MB / 45.9MB, 3.83MB/s  
New Data Upload: 100%|██████████| 45.9MB / 45.9MB, 3.83MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:28<00:00, 28.67s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/justacoderwhocodes/dental_diagnosis_balanced_dataset/commit/b81f147d9e403844b87705d3f4b14c120c7239ad', commit_message='Closeup dataset', commit_description='', oid='b81f147d9e403844b87705d3f4b14c120c7239ad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/justacoderwhocodes/dental_diagnosis_balanced_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='justacoderwhocodes/dental_diagnosis_balanced_dataset'), pr_revision=None, pr_num=None)

In [8]:
check_ds = load_dataset("justacoderwhocodes/dental_diagnosis_balanced_dataset", split="train")

Generating train split: 100%|██████████| 2144/2144 [00:00<00:00, 34349.07 examples/s]


In [9]:
print(check_ds[0])

{'filename': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=448x448 at 0x11DB31C60>, 'tooth': 'first_premolar', 'treatment': 'none', 'diagnosis': 'Tooth appears normal', 'text': 'Tooth appears normal; recommended treatment: none'}
